# Paper Feature Lookup Tool

Quick tool to fetch real AUB papers with all their features for testing the Streamlit app.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_pickle('../data/processed/cleaned_data.pkl')
print(f"Loaded {len(df)} papers")

## Quick Lookup Functions

In [ ]:
def get_paper_features(paper):
    """Extract and format all features for a paper."""
    
    print("=" * 80)
    print("COPY THESE VALUES INTO STREAMLIT APP")
    print("=" * 80)
    print(f"\n📄 TITLE:")
    print(paper['Title'])
    
    print(f"\n📝 ABSTRACT:")
    print(paper['Abstract'][:500] + "..." if len(str(paper['Abstract'])) > 500 else paper['Abstract'])
    
    print(f"\n📅 YEAR: {int(paper['Year'])}")
    
    print(f"\n👁️ VENUE METRICS:")
    print(f"  Views: {int(paper['Views']) if pd.notna(paper['Views']) else 0}")
    print(f"  CiteScore: {paper['CiteScore (publication year)']}")
    print(f"  SJR: {paper['SJR (publication year)']}")
    print(f"  SNIP: {paper['SNIP (publication year)']}")
    
    print(f"\n👥 AUTHOR METRICS:")
    print(f"  Number of Authors: {int(paper['Number of Authors'])}")
    print(f"  Number of Institutions: {int(paper['Number of institutions']) if pd.notna(paper['Number of institutions']) else 1}")
    print(f"  International Collaboration: {'YES' if paper['Number of countries'] > 1 else 'NO'}")
    
    print(f"\n🎯 ACTUAL RESULT:")
    print(f"  Citations: {int(paper['Citations'])} (High-impact: {'YES' if paper['Citations'] >= 26 else 'NO'})")
    print("\n" + "=" * 80)

def search_papers(keyword=None, year=None, min_citations=None, max_citations=None, limit=5):
    """Search papers by criteria."""
    
    filtered = df.copy()
    
    if keyword:
        filtered = filtered[filtered['Title'].str.contains(keyword, case=False, na=False) | 
                          filtered['Abstract'].str.contains(keyword, case=False, na=False)]
    
    if year:
        filtered = filtered[filtered['Year'] == year]
    
    if min_citations is not None:
        filtered = filtered[filtered['Citations'] >= min_citations]
    
    if max_citations is not None:
        filtered = filtered[filtered['Citations'] <= max_citations]
    
    results = filtered.head(limit)
    
    print(f"\nFound {len(filtered)} papers matching criteria. Showing top {min(limit, len(filtered))}:\n")
    
    for idx, (_, paper) in enumerate(results.iterrows(), 1):
        print(f"{idx}. {paper['Title'][:100]}... ({int(paper['Citations'])} citations)")
    
    return results

## Pre-defined Examples

In [ ]:
def get_high_impact_paper():
    """Get a random high-impact paper (top 25%)."""
    high_impact = df[df['Citations'] >= 26].sample(1).iloc[0]
    get_paper_features(high_impact)
    return high_impact

def get_low_impact_paper():
    """Get a random low-impact paper (bottom 75%)."""
    low_impact = df[df['Citations'] < 26].sample(1).iloc[0]
    get_paper_features(low_impact)
    return low_impact

def get_mega_cited_paper():
    """Get a paper with >100 citations."""
    mega = df[df['Citations'] > 100].sample(1).iloc[0]
    get_paper_features(mega)
    return mega

def get_recent_paper():
    """Get a paper from 2019-2020."""
    recent = df[df['Year'].isin([2019, 2020])].sample(1).iloc[0]
    get_paper_features(recent)
    return recent

## Usage Examples

Run any of these cells to get papers for testing:

In [ ]:
get_high_impact_paper()

In [ ]:
get_low_impact_paper()

In [ ]:
get_mega_cited_paper()

In [ ]:
get_recent_paper()

## Search for Specific Papers

In [ ]:
results = search_papers(keyword="machine learning", min_citations=20, limit=5)

In [ ]:
get_paper_features(results.iloc[0])

In [ ]:
results = search_papers(keyword="COVID", year=2020, limit=10)

In [ ]:
get_paper_features(results.iloc[0])

## Get Multiple Examples at Once

In [ ]:
print("\n" + "#" * 80)
print("# EXAMPLE 1: HIGH-IMPACT PAPER")
print("#" * 80)
get_high_impact_paper()

print("\n\n" + "#" * 80)
print("# EXAMPLE 2: LOW-IMPACT PAPER")
print("#" * 80)
get_low_impact_paper()

print("\n\n" + "#" * 80)
print("# EXAMPLE 3: MEGA-CITED PAPER")
print("#" * 80)
get_mega_cited_paper()

## Custom Search Examples

In [ ]:
results = search_papers(keyword="cancer", min_citations=50, year=2018)

In [ ]:
results = search_papers(year=2020, min_citations=0, max_citations=5, limit=10)

## Statistics

In [ ]:
print("Dataset Statistics:")
print(f"Total papers: {len(df)}")
print(f"\nCitation distribution:")
print(f"  Min: {df['Citations'].min():.0f}")
print(f"  25th percentile: {df['Citations'].quantile(0.25):.0f}")
print(f"  Median: {df['Citations'].median():.0f}")
print(f"  75th percentile (high-impact threshold): {df['Citations'].quantile(0.75):.0f}")
print(f"  Mean: {df['Citations'].mean():.1f}")
print(f"  Max: {df['Citations'].max():.0f}")
print(f"\nHigh-impact papers (≥26 citations): {(df['Citations'] >= 26).sum()} ({(df['Citations'] >= 26).sum() / len(df) * 100:.1f}%)")
print(f"\nPapers by year:")
print(df['Year'].value_counts().sort_index())